<a href="https://colab.research.google.com/github/milly-lauren/aiHW2/blob/problem1/hw2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Handcrafted Features


**Load the Data**

In [0]:
from keras.datasets import mnist
import time 

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from keras.utils import to_categorical

#train_labels = to_categorical(train_labels_original)
#test_labels = to_categorical(test_labels_original)

# change the labels to be ints
#train_labels = np.argmax(train_labels, axis=1)
#train_labels = train_labels.astype('uint8')

**Define Features for Average Height and Width of Digits**

This takes a hot minute...


In [0]:
def find_tall(digit):
  for row in range (0,28):
    for col in range(0,28):
      if (digit[row][col]) > 100:
        return row
      
  return 0
      
def find_short(digit):
  for row in range (27, 0 ,-1):
    for col in range(0,28):
      if digit[row][col] > 100:
        return row
       
  return 0

def find_left(digit):
  for col in range (0,28):
    for row in range(0,28):
      if digit[row][col] > 100:
        return col
  return 0

def find_right(digit):
  for col in range (27, 0 ,-1):
    for row in range(0,28):
      if digit[row][col] > 100:
        return col
  return 0  

def find_bounds(digit):
  tall = 28
  short = 0
  left = 28
  right = 0
  for row in range (0,28):
    for col in range (0,28):
      if digit[row][col] > 100:
        if(row < tall):
          tall = row
        if(row > short):
          short = row
        if(col < left):
          left = col
        if(col > right):
          right = col
  
  return tall, short, left, right


In [0]:
num = len(train_images)

start = time.time()

avg_height = np.zeros((10, 1, 1))
avg_width = np.zeros((10,1,1))
freq_digits = np.zeros((10, 1, 1))

# Find the height by finding the "tallest" pixel - "shortest" pixel
# Keep in mind, the image goes from 0 to 28, so the "tallest" is at the smallest row
# Same thing for width, but now the logic follows, the "rightmost" pixel - "leftmost" pixel
# since the array follows like a number line

for idx in np.arange(num):
  label = train_labels[idx]
  digit = train_images[idx]
  #tall, short, left, right = find_bounds(digit)
  avg_height[label] += find_short(digit) - find_tall(digit)
  avg_width[label] += find_right(digit) - find_left(digit)
  freq_digits[label] += 1



avg_height /= freq_digits
avg_width /= freq_digits

end = time.time()

print(end - start)

88.02899384498596


**Normalize the vectors**

In [0]:
hMax = avg_height.max()
avg_height = avg_height / hMax

wMax = avg_width.max()
avg_width = avg_width / wMax


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


**Build the Single Dense Layer Model**
https://stackoverflow.com/questions/43237124/what-is-the-role-of-flatten-in-keras



In [0]:
from keras import models
from keras import layers

# Flatten the images
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

# Concatenate 
test_images = np.concatenate(test_images, avg_height, avg_width)
test_images = np.stack(test_images, avg_height, avg_width)
# Put it through one dense layer
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

TypeError: ignored

In [0]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

epochs = 10
history = model.fit(train_images, 
                      train_labels, 
                      epochs=epochs,  
                      validation_data=(test_images, test_labels))

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

In [0]:
import matplotlib.pyplot as plt
import numpy as np

history_dict = history.history
loss_values = history_dict['loss']
test_loss_values = history_dict['val_loss']
epochs_range = range(1, epochs+1)

plt.plot(epochs_range, loss_values, 'go', label='Training loss')
plt.plot(epochs_range, test_loss_values, 'ro', label='Test loss')
plt.title('Training and test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

make copy then convert to 0 and 1

image > 100 = 1 

image < 100 = 0
